In [ ]:
# training_model.ipynb
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import mlflow
import mlflow.keras

In [ ]:
# Charger les données
file_path = "data/energy_efficiency_data.csv"
df = pd.read_csv(file_path)

# Séparer features et labels
X = df.drop(columns=["Heating_Load", "Cooling_Load"]).values
y = df[["Heating_Load", "Cooling_Load"]].values


In [ ]:
# Normalisation
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42
)


In [ ]:
# Création du modèle ANN
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(2, activation="linear")
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# MLflow
mlflow.set_experiment("Energy_Efficiency_ANN")
with mlflow.start_run():
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=100,
        batch_size=32,
        verbose=1
    )

    mlflow.keras.log_model(model, artifact_path="model")
    mlflow.log_param("epochs", 100)
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("optimizer", "adam")

    # Évaluation
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    mlflow.log_metric("mae", mae)
    mlflow.log_metric("mse", mse)

    print(f"Test MAE: {mae:.4f}, Test MSE: {mse:.4f}")


In [ ]:
# Sauvegarde du modèle et scalers
model.save("models/Energy_Efficiency_Model.h5")

from joblib import dump
dump(scaler_X, "models/scaler_X.pkl")
dump(scaler_y, "models/scaler_y.pkl")

In [ ]:
# Visualisation
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.legend()
plt.title("Courbes de perte")
plt.xlabel("Epochs")
plt.ylabel("MSE Loss")
plt.show()

In [ ]:
# Prédictions vs réel
y_pred_rescaled = scaler_y.inverse_transform(y_pred)
y_test_rescaled = scaler_y.inverse_transform(y_test)

plt.figure(figsize=(6,6))
plt.scatter(y_test_rescaled[:,0], y_pred_rescaled[:,0], alpha=0.7)
plt.plot([y_test_rescaled[:,0].min(), y_test_rescaled[:,0].max()],
         [y_test_rescaled[:,0].min(), y_test_rescaled[:,0].max()],
         color="red", linestyle="--")
plt.xlabel("Valeurs réelles (Heating_Load)")
plt.ylabel("Prédictions ANN")
plt.title("Prédictions vs Réel - Heating Load")
plt.show()
